In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os, time, random
from bs4 import BeautifulSoup
import urllib.request


def chromeWebdriver():
    options = Options()
    options.add_argument("lang=ko_KR")  # 언어 설정
    # options.add_argument("start-maximized") # 창 크기 최대로
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")    
    options.add_experimental_option('detach', True)  # 브라우저 안 닫히게
    options.add_experimental_option('excludeSwitches', ['enable-logging'])  # 시스템 장치 에러 숨기기
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')    
    # options.add_argument('--headless')  # 웹 브라우저를 시각적으로 띄우지 않는 headless chrome 옵션
    driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()), options=options)
    return driver


def collect_image(search_word, extract_img_count):
    url = 'https://www.google.co.kr'

    now = time.localtime()
    today_time = f'{now.tm_year}{now.tm_mon}{now.tm_mday}_{now.tm_hour}{now.tm_min}'
    print(today_time)

    file_path = "C:/Python/instagram img crawling/img"

    os.chdir(file_path)
    os.makedirs(file_path + today_time + '_' + search_word)
    os.chdir(file_path + today_time + '_' + search_word)
    file_save_dir = file_path + today_time + '_' + search_word
    print(file_save_dir)

    driver = chromeWebdriver()
    driver.get(url)
    time.sleep(random.uniform(2, 3))
    elem_q = driver.find_element(By.NAME, 'q')
    elem_q.send_keys(search_word)
    elem_q.submit()

    driver.find_element(By.LINK_TEXT, '이미지').click()  # 텍스트 메뉴 '이미지' 링크 클릭
    # driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
    time.sleep(random.uniform(1, 2))

    # 페이지 스크롤 다운
    def page_scrolling(drivers):
        ## scrolling ------------------------------
        elem = driver.find_element(By.TAG_NAME, 'body')
        page_height = driver.execute_script('return document.body.scrollHeight')
        # print(page_height)

        # more_view_cnt = 0
        scroll_cnt = 1
        more_view_scroll_cnt = -1  # '결과 더보기' 버튼 나올 때의 scroll_cnt (break 처리 위해 사용)
        equal_cnt = 1
        while True:
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(random.uniform(0.3, 0.5))
            new_height = driver.execute_script('return document.body.scrollHeight')
            if page_height != new_height:
                page_height = new_height
                equal_cnt = 1
            print(f'scroll_cnt: {scroll_cnt}, new_height: {new_height}, equal_cnt: {equal_cnt}')
            
            try:
                scroll_cnt += 1
                equal_cnt += 1
                driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()  # 결과 더보기 버튼 처리
                print('결과 더보기 버튼 클릭 처리')
                more_view_scroll_cnt = scroll_cnt
                more_view_cnt += 1
            except:
                if equal_cnt == 20:  # scroll_cnt / more_view_scroll_cnt > 2.5:
                    break
                continue
        ## End of scrolling ------------------------------

    page_scrolling(driver)

    file_no = 1
    count = 1
    img_src = []

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    # imgs = driver.find_elements(By.TAG_NAME, 'img')
    imgs = driver.find_elements(By.CSS_SELECTOR, '#islrg > div.islrc > div a.wXeWr.islib.nfEiy')
    print(len(imgs))

    for img in imgs:
        img_src1 = img.click()  # 이미지 클릭 시 display 되는 url을 찾기 위해 클릭함
        try:
            img_src2 = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div > a')
        except Exception:
            continue
        time.sleep(random.uniform(0.2, 0.5))
        img_src3 = img_src2.find_element(By.TAG_NAME, 'img').get_attribute('src')
        if img_src3[:4] != 'http':
            continue
        print(count, img_src3, '\n')

        img_src.append(img_src3)
        if count == extract_img_count + 10:  # 이미지 에러 대비해서 입력 숫자보다 크게 잡음
            break        
        count += 1
        
    print(f'\n{"="*10} 추출한 전체 리스트 {"="*10}\n{img_src}\n\n{"="*10}총 {len(img_src)}개 추출함{"="*10}\n')

    for i in range(len(img_src)):
        extention = img_src[i].split('.')[-1]
        ext = ''
        if extention in ('jpg', 'JPG', 'jpeg', 'JPEG', 'png', 'PNG', 'gif', 'GIF'):
            ext = '.' + extention
        else:
            ext = '.jpg'        
        try:
            urllib.request.urlretrieve(img_src[i], str(file_no).zfill(3) + ext)
            print(img_src[i])
        except Exception:
            continue

        print(f'{file_no}번째 이미지 저장-----')
        file_no += 1
        
        if file_no - 1 == extract_img_count:
            break

    driver.close()


if __name__ == '__main__':
    collect_image('민트 초콜릿 칩 블렌디드', 200)

2022822_2334
C:/Python/instagram img crawling/img2022822_2334_민트 초콜릿 칩 블렌디드
scroll_cnt: 1, new_height: 3204, equal_cnt: 1
scroll_cnt: 2, new_height: 6358, equal_cnt: 1
scroll_cnt: 3, new_height: 6358, equal_cnt: 2
scroll_cnt: 4, new_height: 6358, equal_cnt: 3
scroll_cnt: 5, new_height: 6358, equal_cnt: 4
scroll_cnt: 6, new_height: 6358, equal_cnt: 5
scroll_cnt: 7, new_height: 6358, equal_cnt: 6
scroll_cnt: 8, new_height: 12666, equal_cnt: 1
scroll_cnt: 9, new_height: 12666, equal_cnt: 2
scroll_cnt: 10, new_height: 12666, equal_cnt: 3
scroll_cnt: 11, new_height: 12666, equal_cnt: 4
scroll_cnt: 12, new_height: 12666, equal_cnt: 5
scroll_cnt: 13, new_height: 12666, equal_cnt: 6
scroll_cnt: 14, new_height: 12666, equal_cnt: 7
scroll_cnt: 15, new_height: 12666, equal_cnt: 8
scroll_cnt: 16, new_height: 12666, equal_cnt: 9
scroll_cnt: 17, new_height: 18490, equal_cnt: 1
scroll_cnt: 18, new_height: 18490, equal_cnt: 2
scroll_cnt: 19, new_height: 18490, equal_cnt: 3
scroll_cnt: 20, new_height: 

51 https://d2uja84sd90jmv.cloudfront.net/posts/pVFYQ56WQxcxrp3zMLverQ/m.jpg 

52 https://img.siksinhot.com/article/1621384905675199.jpeg 

53 https://img2.joongna.com/cafe-article-data/live/2022/07/20/930642605/1658291214978000ZGi_CSzxv.jpg?impolicy=resizeWatermark3&ftext=%EB%B0%B1%EC%B0%A8%EB%8B%88 

54 http://img1.tmon.kr/cdn4/deals/2022/03/22/2585637242/2585637242_front_33f29_c9f8b.jpg 

55 http://img.danawa.com/prod_img/500000/440/562/img/3562440_1.jpg?_v=20191022163647 

56 https://post-phinf.pstatic.net/MjAyMDExMDlfMTAg/MDAxNjA0ODg0MjEwMzQ1.fvMb0t6aDkY9JivGBFZ29mzEO-vGD6Hgx_5VE-CL3-gg.-gq7Qnq89gKJBTAphA4Pwr5Gg6eQ7__rwXIPLHRG4jYg.PNG/2.png?type=w1200 

57 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRzl87WIhcOe9NQ45n1EemAwyaE-sz2wkqNVA&usqp=CAU 

58 https://img.extmovie.com/files/attach/images/148/647/445/065/2030abcba9cd557263496f05012ff3c1.jpg 

59 https://media.bunjang.co.kr/product/156537459_1_1623471356_w%7Bres%7D.jpg 

60 https://todaycoffee.kr/data/editor/2105/2021

134 https://blog.kakaocdn.net/dn/cye8eD/btq4ZRj1Gjm/YDm729KSb6aSeGFEFjKgu0/img.jpg 

135 https://blog.kakaocdn.net/dn/vgnyd/btrcM0VN0m6/57zuS2nkkEKUjl3VRPzqS0/img.jpg 

136 https://shop2.daumcdn.net/thumb/R500x500/?fname=http%3A%2F%2Fshop2.daumcdn.net%2Fshophow%2Fp%2FM11170901529.jpg%3Fut%3D20220328163746 

137 https://img4.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202105/06/cosmopolitan/20210506180415393gtjq.jpg 

138 https://image.istarbucks.co.kr/upload/store/skuimg/2022/03/[9200000003997]_20220311105602098.jpg 

139 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBn8hsiiRmwow5JG5hCUUcSxeFY21OhWMUMg&usqp=CAU 

140 https://sitem.ssgcdn.com/16/74/87/item/1000052877416_i1_500.jpg 

141 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWsRPMWa4W7i5PkntkKvrvwhtOl34DCd5OIA&usqp=CAU 

142 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJEbgFcX9vBhcW3_grSZ3ISFITaprLmvbREQ&usqp=CAU 

143 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwCzCOYRoF

https://blog.kakaocdn.net/dn/rzz42/btq414J0WJe/KYYtEwUZQTekOXvtocg1d1/img.png
1번째 이미지 저장-----
https://sitem.ssgcdn.com/93/95/39/item/1000326399593_i1_1200.jpg
2번째 이미지 저장-----
https://mblogthumb-phinf.pstatic.net/MjAyMTA0MzBfNTEg/MDAxNjE5Nzc1NTU2ODU5.cBbtaLo_Z804YII7fG6i86i570DsdOx1wMooAvUgYJQg.LCNbVbIXp1koOThlnolaqa1-5dH1RnGbXqHlWdFK_bYg.JPEG.o_chenny_o/IMG_0309.jpg?type=w800
3번째 이미지 저장-----
https://mblogthumb-phinf.pstatic.net/MjAyMTA1MDhfMjUg/MDAxNjIwNDcxNzg5MDg0.X9GKGRT-XX8B59YkSKXYPlTtsn5waNbRTFzLTB4aqQ4g.scUkbTzryLz1o9NJ3WrJ9I-4cPXizcVeJQCnP6Ya438g.JPEG.vvgmvg/output_3148140217.jpg?type=w800
4번째 이미지 저장-----
https://blog.kakaocdn.net/dn/bHwo33/btq8kKiwuSf/IKU2GhKNnZCZm9V2fh25WK/img.png
5번째 이미지 저장-----
https://img.siksinhot.com/article/1621384838127195.jpeg
6번째 이미지 저장-----
https://blog.kakaocdn.net/dn/dfHizR/btq4vRrqeOW/osHFupDGRL0b4QZKKHqj11/img.png
7번째 이미지 저장-----
https://blog.kakaocdn.net/dn/VJgy2/btq3UcYk56x/rQboraRtWoIB4LNxBfvsRK/img.jpg
8번째 이미지 저장-----
https://mblogthumb-phinf

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGqyr-jCOGHuCbzS0KJ2u8Emo05bO-wgK2hA&usqp=CAU
71번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRRGfyTslU2J-CqMVvqWC13eFnTnE0HFmWKfQ&usqp=CAU
72번째 이미지 저장-----
https://img.siksinhot.com/article/1621384888701226.jpeg
73번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRBddY1_jidklCDmjKZpuC5cFF4ZK6m_MfyMg&usqp=CAU
74번째 이미지 저장-----
https://cdnweb01.wikitree.co.kr/webdata/editor/202104/28/img_20210428172712_0f670ebe.webp
75번째 이미지 저장-----
https://blog.kakaocdn.net/dn/bgUuY2/btrcp02skms/MYOiJvqi5NkbQkK60Pcjs1/img.png
76번째 이미지 저장-----
https://cdn.cashfeed.co.kr/attachments/62f8d70568.jpg
77번째 이미지 저장-----
https://image.ajunews.com/content/image/2021/06/03/20210603100642152389.jpg
78번째 이미지 저장-----
https://img2.joongna.com/media/original/2022/07/25/1658760722616ZK4_ytbvv.jpg?impolicy=thumb&size=150
79번째 이미지 저장-----
https://cdn.011st.com/11dims/strip/false/11src/ui_img/contents/0607/3/0.jpg
80번째 이미지 저장---

https://img.etoday.co.kr/pto_db/2021/05/600/20210503225006_1615898_937_461.PNG
145번째 이미지 저장-----
https://cdn.singleliving.kr/singleliving/2021/05/02-8.jpg
146번째 이미지 저장-----
https://img.siksinhot.com/article/1621384922126229.jpeg
147번째 이미지 저장-----
https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FmxMI6%2Fbtra9WmAVTK%2FLE4RkJRZuzLsIL2XC1k0t1%2Fimg.jpg
148번째 이미지 저장-----
https://i.ytimg.com/vi/nFlYaqbdhKM/maxresdefault.jpg
149번째 이미지 저장-----
https://image.istarbucks.co.kr/upload/store/skuimg/2022/07/[9200000003816]_20220701141004383.jpg
150번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQGBOviTghvIIYOFJpAeUVKNflDm4hmD01OWw&usqp=CAU
151번째 이미지 저장-----
https://img4.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202105/24/siksin/20210524040135155eqaf.jpg
152번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTflX2EbWkig9OVMAGzjtQn7M2kBRp0EQm5HQ&usqp=CAU
153번째 이미지 저장-----
https://encrypted-tb